In [1]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path = '/content/drive/MyDrive/전력수요예측/'

In [3]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# train 데이터 불러오기 (더미 x)

In [4]:
# train 파일 불러오고 전처리
train = pd.read_csv(path+'electric_train_cp949.csv')
train.drop('Unnamed: 0', axis=1, inplace=True)
cols = [col[(col.find('.')+1):] for col in train.columns]
train.columns = cols

# datetime 형식으로 변환
train['tm'] = pd.to_datetime(train['tm'])

# 컬럼 추가
train['year'] = train['tm'].dt.year
train['month'] = train['tm'].dt.month
train['day'] = train['tm'].dt.day

# train 데이터의 elec 이상치 처리
for t in list(train.loc[train['elec'] < 0, 'nph_ta']):
    train.loc[(train['elec'] < 0)&(train['nph_ta'] == t), 'elec']\
    = train.loc[(train['nph_ta'] == t)&(train['elec'] >= 0), :]['elec'].mean()

In [5]:
# test 파일 불러오고 전처리
test = pd.read_csv(path+'electric_test.csv')
test.columns = ['num', 'tm', 'hh24', 'stn', 'nph_ta', 'nph_hm', 'nph_ws_10m',
       'nph_rn_60m', 'nph_ta_chi', 'weekday', 'week_name', 'elect']
test.drop(['elect'] ,axis=1, inplace=True)

# datetime 형식으로 변환
test['tm'] = pd.to_datetime(test['tm'])

# 컬럼 추가
test['year'] = test['tm'].dt.year
test['month'] = test['tm'].dt.month
test['day'] = test['tm'].dt.day
test.drop('tm', axis=1, inplace=True)

In [6]:
X = train.drop(['tm', 'n', 'n_mean_load', 'sum_qctr', 'sum_load', 'elec'], axis=1)
Y = train.loc[:,['elec']]

print(X.columns)
print(test.columns)

Index(['num', 'hh24', 'stn', 'nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m',
       'nph_ta_chi', 'weekday', 'week_name', 'year', 'month', 'day'],
      dtype='object')
Index(['num', 'hh24', 'stn', 'nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m',
       'nph_ta_chi', 'weekday', 'week_name', 'year', 'month', 'day'],
      dtype='object')


In [7]:
from sklearn.preprocessing import MinMaxScaler
X_cols = X.columns
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X =pd.DataFrame(X)
X.columns = X_cols

test_cols = test.columns
test = scaler.transform(test)
test =pd.DataFrame(test)
test.columns = test_cols

print()
y_max, y_min = Y.max(), Y.min()
Y = (Y-y_min) / (y_max-y_min)

print(y_max)
print(y_min)


elec    373.06
dtype: float64
elec    48.9
dtype: float64


In [8]:
train = pd.merge(X,Y,left_index=True, right_index=True)

In [ ]:
display(.head())
display(test.head())

,num,tm,hh24,n,stn,sum_qctr,sum_load,n_mean_load,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elec,year,month,day
0,4821,2021-01-01 01:00:00,1,11,884,6950,751.32,68.606449,2.2,62.7,1.8,0.0,-1.0,4,0,99.56,2021,1,1
1,4821,2021-01-01 02:00:00,2,11,884,6950,692.60,68.606449,2.3,63.1,2.1,0.0,-0.6,4,0,91.78,2021,1,1
2,4821,2021-01-01 03:00:00,3,11,884,6950,597.48,68.606449,2.2,62.4,2.5,0.0,-1.3,4,0,79.17,2021,1,1
3,4821,2021-01-01 04:00:00,4,11,884,6950,553.48,68.606449,1.7,63.5,1.7,0.0,-0.2,4,0,73.34,2021,1,1
4,4821,2021-01-01 05:00:00,5,11,884,6950,526.24,68.606449,1.7,63.0,1.6,0.0,-0.8,4,0,69.73,2021,1,1


,num,hh24,stn,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,year,month,day
0,-0.15106,0.000000,0.229563,0.526829,0.604317,0.836234,0.0,0.482059,1.0,1.0,1.0,0.0,0.0
1,-0.15106,0.043478,0.229563,0.528455,0.595067,0.843354,0.0,0.478939,1.0,1.0,1.0,0.0,0.0
2,-0.15106,0.086957,0.229563,0.533333,0.608428,0.852057,0.0,0.480499,1.0,1.0,1.0,0.0,0.0
3,-0.15106,0.130435,0.229563,0.536585,0.598150,0.856013,0.0,0.483619,1.0,1.0,1.0,0.0,0.0
4,-0.15106,0.173913,0.229563,0.538211,0.598150,0.858386,0.0,0.483619,1.0,1.0,1.0,0.0,0.0


## timesteps = 24

In [ ]:
timesteps = 24

x_cols = ['num', 'hh24', 'stn', 'nph_ta', 'nph_hm', 'nph_ws_10m',
          'nph_rn_60m', 'nph_ta_chi', 'weekday', 'week_name', 'year', 'month', 'day']
y_cols = ['elec']

In [ ]:
# train data temporalize
x = []
for num in tqdm(train['num'].unique()):
    num_x = train.loc[train['num'] == num, x_cols]
    for i in range(len(num_x) - timesteps + 1):
        x.append(num_x.iloc[i:i+timesteps])
X = np.array(x)
del x

y = []
for num in tqdm(train['num'].unique()):
    num_y = train.loc[train['num'] == num, y_cols]
    for i in range(len(num_y) - timesteps + 1):
        y.append(num_y.iloc[i+timesteps-1])

Y = np.array(y)
del y, num_x, num_y

print(X.shape)
print(Y.shape)

100%|██████████| 323/323 [04:29<00:00,  1.20it/s]


(7585926, 24, 13)
(7585926, 1)


In [ ]:
np.save(path+'X_train_24.npy',X)
np.save(path+'Y_train_24.npy',Y)
del X, Y

In [ ]:
# test data temporalize
T = []
for num in tqdm(test['num'].unique()):
    num_x = test.loc[test['num'] == num, x_cols]
    for i in range(len(num_x) - timesteps + 1):
        T.append(num_x.iloc[i:i+timesteps])
T = np.array(T)

print(T.shape)

100%|██████████| 324/324 [01:24<00:00,  3.85it/s]


(2830787, 24, 13)


In [ ]:
np.save(path+'test_24.npy',T)

In [ ]:
# 다운로드 함수
import gdown, zipfile, os, glob, shutil
def download_file(file_id, save_path) :
    if os.path.exists(save_path) :
        print(f'{save_path} 파일이 이미 존재합니다.')
        return

    gdown.download(id=file_id, output=save_path, quiet=False)

#위의 전처리된 3개 파일 다운로드 함수
download_file('1Jy25yk5tFvZnbVyZa5y5o9_IMsbw5GCC', '.X_train_24.npy')
download_file('1tzWAyQ6AjBGDvb9VkgKx8ELLKZbnbSZc', '.Y_train_24.npy')
download_file('1-4Z1eNwr1pumiHdHVT1hyqwlxVDUKYa_', '.test_24.npy')